**ESTE NOTEBOOK TIENE SU ORIGEN EN GOOGLE COLAB**

Este notebook es un fine-tunned hecho a un modelo de BERT para predecir ratings a partir de un dataset de reviews.

Logro una exactitud del 88% en test dataset. Predice ratings entre 1 a 5.

<hr>

**DESCARGAMOS LAS LIBRERIAS A UTILIZAR**

In [5]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00

<hr>

**CARGAMOS LA DATA DESDE GOOGLE DRIVE**

Para ello, debemos hacer la conexion o haber montado antes.

In [11]:
from datasets import load_from_disk

In [12]:
data = load_from_disk(r'/content/drive/MyDrive/datasets/test_data')

<hr>

**MODELO**

AutoTokenizer es una funcion que carga el tokenizer del modelo, y convierte nuestras reviews en inputs para el modelo.


DataCollatorWithPadding es una funcion que realiza un padding dinamico para reducir el tamaño de nuestros vectores.

TFAutoModelForSequenceClassification agrega una cabeza al transformer para la tarea de clasificacion. En el fondo, crea layers.

In [36]:
import tensorflow as tf

from transformers import AutoTokenizer

from transformers import DataCollatorWithPadding
from transformers import TFAutoModelForSequenceClassification

# Funcion de perdida para problema de clasificacion.
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [15]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**CREAMOS LOS DATASET PARA EL MODELO**

In [16]:
tf_train_dataset = data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["rating"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = data["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["rating"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


**EXTRA**

PolynomialDecay decrece en el runtime la tasa de aprendizaje. Esto impacta en el performance del modelo alrededor de un 27%

In [17]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

In [18]:
batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)

In [19]:
# Usamos accuracy como metrica de evaluacion.
model.compile(
    optimizer=opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"])

In [20]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 109,486,085
Trainable params: 109,486,085
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Entranamos en 3 epocas.
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3,
    verbose=True,
)

Epoch 1/3
6380/6380 [==============================] - 1372s 207ms/step - loss: 0.7074 - accuracy: 0.7272 - val_loss: 0.5842 - val_accuracy: 0.7794
Epoch 2/3
6380/6380 [==============================] - 1208s 189ms/step - loss: 0.5861 - accuracy: 0.7674 - val_loss: 0.4629 - val_accuracy: 0.8167
Epoch 3/3
6380/6380 [==============================] - 1209s 190ms/step - loss: 0.4425 - accuracy: 0.8231 - val_loss: 0.3271 - val_accuracy: 0.8833


In [25]:
model.config.label2id = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4}

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4}

In [26]:
# Guardamos el modelo.
model.save_pretrained('/content/drive/MyDrive/datasets/bert_classification_alaska_88')

**EVALUATION**

In [35]:
import sklearn.metrics as skm
import numpy as np

In [58]:
y_pred = model.predict(tf_validation_dataset)
y_pred = np.argmax(tf.math.softmax(y_pred['logits'], axis=-1), axis=1)
y_true = np.array(data['validation']['rating'])

score = skm.accuracy_score(y_true=y_true, y_pred=y_pred)
print(f'Accuracy for the validation dataset: {score:.2f}')

score = skm.f1_score(y_true=y_true, y_pred=y_pred, average='macro')
print(f'F1-score for the validation dataset: {score:.2f}')


128/128 [==============================] - 11s 81ms/step
Accuracy for the validation dataset: 0.88
F1-score for the validation dataset: 0.81


In [56]:
tf_test_dataset = data["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["rating"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [59]:
y_pred = model.predict(tf_test_dataset)
y_pred = np.argmax(tf.math.softmax(y_pred['logits'], axis=-1), axis=1)
y_true = np.array(data['test']['rating'])

score = skm.accuracy_score(y_true=y_true, y_pred=y_pred)
print(f'Accuracy for the test dataset: {score:.2f}')

score = skm.f1_score(y_true=y_true, y_pred=y_pred, average='macro')
print(f'F1-score for the test dataset: {score:.2f}')

511/511 [==============================] - 37s 72ms/step
Accuracy for the test dataset: 0.88
F1-score for the test dataset: 0.80


**EJEMPLOS**

In [75]:
example_review = ['The service was really good.', 'Horrible horrible horrible, I will never come back again.', 'The food was okey, but the waiter was very disrespectful.']
example_inputs = tokenizer(example_review, padding=True, truncation=True, return_tensors='tf')
example_pred = np.argmax(model(**example_inputs)['logits'], axis=1)

for i in range(len(example_review)):
    print(f'Review: {example_review[i]} | Rating: {example_pred[i]}')

Review: The service was really good. | Rating: 3
Review: Horrible horrible horrible, I will never come back again. | Rating: 0
Review: The food was okey, but the waiter was very disrespectful. | Rating: 1
